In [1]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GraphSAGE
import torch.nn.functional as F

import importlib

import training
importlib.reload(training)
import graph_vis
importlib.reload(graph_vis)
import graph_creation
importlib.reload(graph_creation)
import utils
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\csaba\\Documents\\Coding\\git_own\\thesis_coding\\utils.py'>

In [2]:
device = utils.set_seeds_and_device() 
dataset,data = training.load_dataset('Cora', "./training_data/datasets")


Using device: cpu
Random seed set to: 42


In [3]:
# Train function
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    # cross entroy loss makes this model supervised 
    loss.backward()
    optimizer.step()
    return loss.item()

# Test function
@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask] == data.y[mask]
        accs.append(int(correct.sum()) / int(mask.sum()))
    return accs


In [4]:
# Define the GraphSAGE model
model = GraphSAGE(
    in_channels=dataset.num_features,
    hidden_channels=64,
    num_layers=4,
    out_channels=dataset.num_classes,
).to(device)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 101):
    loss = train()
    train_acc, val_acc, test_acc = test()
    print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, '
          f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch 001, Loss: 1.9492, Train: 0.6643, Val: 0.4780, Test: 0.4940
Epoch 002, Loss: 1.8872, Train: 0.6786, Val: 0.4860, Test: 0.4830
Epoch 003, Loss: 1.6736, Train: 0.7571, Val: 0.5900, Test: 0.6070
Epoch 004, Loss: 1.2094, Train: 0.8714, Val: 0.6200, Test: 0.6490
Epoch 005, Loss: 0.6999, Train: 0.9571, Val: 0.6120, Test: 0.6330
Epoch 006, Loss: 0.3930, Train: 0.9357, Val: 0.7120, Test: 0.7470
Epoch 007, Loss: 0.2646, Train: 0.9857, Val: 0.7100, Test: 0.7240
Epoch 008, Loss: 0.0930, Train: 1.0000, Val: 0.7200, Test: 0.7130
Epoch 009, Loss: 0.0293, Train: 0.9929, Val: 0.7220, Test: 0.7180
Epoch 010, Loss: 0.0179, Train: 1.0000, Val: 0.6920, Test: 0.7110
Epoch 011, Loss: 0.0108, Train: 1.0000, Val: 0.6940, Test: 0.7110
Epoch 012, Loss: 0.0076, Train: 1.0000, Val: 0.6980, Test: 0.7180
Epoch 013, Loss: 0.0024, Train: 1.0000, Val: 0.6900, Test: 0.7150
Epoch 014, Loss: 0.0006, Train: 1.0000, Val: 0.6940, Test: 0.7230
Epoch 015, Loss: 0.0003, Train: 1.0000, Val: 0.7040, Test: 0.7270
Epoch 016,

In [5]:
# 3 aggrs, at the end create illsutration and get embeddings
for i in ['mean','max','lstm']:
    # Define the GraphSAGE model
    model = GraphSAGE(
        in_channels=dataset.num_features,
        hidden_channels=64,
        num_layers=2,
        aggr=i,
        out_channels=dataset.num_classes,
    ).to(device)

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(1, 201):
        loss = train()
        train_acc, val_acc, test_acc = test()
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, '
            f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

    model.eval() # Set model to evaluation mode
    with torch.no_grad(): # Disable gradient calculation
        final_embeddings = model(data.x, data.edge_index).cpu().numpy() # Get embeddings for all nodes and move to CPU/NumPy
    print(f"Final node embeddings shape: {final_embeddings.shape}") # Print the shape of the final embeddings
    print(f"Final node embeddings: {final_embeddings}") # Print the final node embeddings

    print("Training complete!" )

Epoch 001, Loss: 1.9536, Train: 0.9929, Val: 0.6520, Test: 0.6460
Epoch 002, Loss: 1.5785, Train: 0.9929, Val: 0.7120, Test: 0.7120
Epoch 003, Loss: 1.0439, Train: 0.9929, Val: 0.7320, Test: 0.7250
Epoch 004, Loss: 0.5554, Train: 0.9929, Val: 0.7500, Test: 0.7510
Epoch 005, Loss: 0.2430, Train: 1.0000, Val: 0.7680, Test: 0.7670
Epoch 006, Loss: 0.0936, Train: 1.0000, Val: 0.7640, Test: 0.7720
Epoch 007, Loss: 0.0351, Train: 1.0000, Val: 0.7640, Test: 0.7710
Epoch 008, Loss: 0.0135, Train: 1.0000, Val: 0.7680, Test: 0.7720
Epoch 009, Loss: 0.0055, Train: 1.0000, Val: 0.7700, Test: 0.7760
Epoch 010, Loss: 0.0024, Train: 1.0000, Val: 0.7640, Test: 0.7760
Epoch 011, Loss: 0.0011, Train: 1.0000, Val: 0.7580, Test: 0.7720
Epoch 012, Loss: 0.0006, Train: 1.0000, Val: 0.7540, Test: 0.7700
Epoch 013, Loss: 0.0003, Train: 1.0000, Val: 0.7500, Test: 0.7700
Epoch 014, Loss: 0.0002, Train: 1.0000, Val: 0.7460, Test: 0.7680
Epoch 015, Loss: 0.0001, Train: 1.0000, Val: 0.7460, Test: 0.7630
Epoch 016,


KeyboardInterrupt

